In [1]:
import sys
import os
from accelerate import notebook_launcher
from datasets import load_dataset
import logging

# Configure logging:
# (i) lear any pre-existing logging handlers (especially useful in notebook reruns)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
# (ii) now safely configure logging
logging.getLogger("codecarbon").setLevel(logging.ERROR)
logging.basicConfig(level=logging.INFO)

# Adjust paths -> import classes
project_root = os.getcwd()  
if project_root not in sys.path:
    sys.path.append(project_root)
from classes.experiment_config import ExperimentConfig
from classes.experiment_runner import ExperimentRunner

#import torch.multiprocessing as mp
#mp.set_start_method('spawn', force=True)

In [1]:
import subprocess

def cleanup_port(port):
    try:
        # Get a list of process IDs using the port.
        result = subprocess.check_output(f"lsof -ti :{port}", shell=True)
        pids = result.decode("utf-8").strip().split("\n")
        if not pids or pids == ['']:
            print(f"No processes found on port {port}.")
            return
        print(f"Found processes on port {port}: {pids}")
        for pid in pids:
            # Get process details (if necessary) to verify it's safe to kill.
            details = subprocess.check_output(f"ps -p {pid} -o comm=", shell=True)
            process_name = details.decode("utf-8").strip()
            print(f"Process {pid} running: {process_name}")
            # Use SIGTERM first (default signal) rather than SIGKILL.
            print(f"Terminating PID {pid} with SIGTERM")
            subprocess.run(f"kill {pid}", shell=True)
        print("Cleanup complete.")
    except subprocess.CalledProcessError as e:
        print(f"No processes found on port {port} or error occurred: {e}")


In [ ]:
experiment_config = ExperimentConfig(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    is_encoder_decoder="decoder_only",
    task_type="text_generation",
    inference_type="pure_generative",  
    max_input_tokens=512,
    max_output_tokens=50,
    num_input_prompts=5,
    save_outputs=True,
    decode_token_to_text=True,
    gpu_list=[0,1,2,3],
    num_processes=2,
    batching_options={
        "fixed_max_batch_size": 2, # this is max batch size if adaptive batching on; fixed batch size if it's off
        "adaptive_batching": False,
        "adaptive_max_tokens": 512  
    },
    sharding_config={
        "fsdp_config": {
            "use_orig_params": True,
            "cpu_offload": True
        },
        "sharding_strategy": "NO_SHARD"
    },
    query_rate=1,
    decoder_temperature=1.0, # NB: needs to be a float
    fp_precision="float32",
    backend="pytorch"
)

# Load prompts (here, using the 'arxiv' split from the lighteval/pile_helm dataset)
ds = load_dataset("lighteval/pile_helm", "arxiv")["test"]
prompts = [sample["text"] for sample in ds]

# function to run the whole experiment workflow.
def run_experiment_workflow():
    # run experiment
    runner = ExperimentRunner(experiment_config, prompts)
    runner.run_torch()
    
    # If not main process, exit cleanly after run
    if not runner.accelerator.is_main_process:
        return  # do NOT proceed further

    # Only main process continues
    if runner.accelerator.is_main_process:
         # aggregate results
        runner.aggregate_results()
        # save results
        runner.save_experiment_results()  
        
    # could add in script here to delete JSON files if storage an issue
    
    runner.teardown()

    return runner

# Launch the experiment across the specified number of processes.
notebook_launcher(run_experiment_workflow,
                    num_processes=experiment_config.num_processes) 
                    #terminate_on_error=True) # USE THIS IN accelerate.launch.launch()                 

Launching training on 2 GPUs.
Found processes on port 29500: ['2022314', '2022473', '2022476']
Killing PID 2022314Found processes on port 29500: ['2022314', '2022473', '2022476']

